# Python相似度计算
https://blog.csdn.net/Yellow_python/article/details/81069692

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')  # 不打印警告
import numpy as np
from pandas import DataFrame

# 语料

In [2]:
poem1 = '''《将进酒》——李白
君不见黄河之水天上来，奔流到海不复回。君不见高堂明镜悲白发，朝如青丝暮成雪。
人生得意须尽欢，莫使金樽空对月。天生我材必有用，千金散尽还复来。
烹羊宰牛且为乐，会须一饮三百杯。岑夫子，丹丘生，将进酒，杯莫停。与君歌一曲，请君为我倾耳听。
钟鼓馔玉不足贵，但愿长醉不复醒。古来圣贤皆寂寞，惟有饮者留其名。
陈王昔时宴平乐，斗酒十千恣欢谑。主人何为言少钱，径须沽取对君酌。
五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。'''.replace('\n', '')
poem2 = '''《惜樽空》——李白
君不见黄河之水天上来，奔流到海不复回。君不见床头明镜悲白发，朝如青云暮成雪。
人生得意须尽欢，莫使金樽空对月。天生吾徒有俊才，千金散尽还复来。
烹羊宰牛且为乐，会须一饮三百杯。岑夫子，丹丘生，与君哥一曲，请君为我倾。
钟鼓玉帛岂足贵，但用长醉不复醒。古来贤圣皆死尽，惟有饮者留其名。
陈王昔时宴平乐，斗酒十千恣欢谑。主人何为言少钱，径须沽取对君酌。
五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。'''.replace('\n', '')
ls1, ls2 = list(poem1), list(poem2)

# 交集 / 并集

In [3]:
s1, s2 = set(ls1), set(ls2)
len(s1 & s2) / len(s1 | s2)

0.8311688311688312

# 编辑距离

In [4]:
def edit_distance_matrix(s1, s2):
    l1, l2 = len(s1) + 1, len(s2) + 1
    matrix = [[0 for j in range(l2)] for i in range(l1)]
    for i in range(l1):
        matrix[i][0] = i
    for j in range(l2):
        matrix[0][j] = j
    for i in range(1, l1):
        for j in range(1, l2):
            delta = 0 if s1[i - 1] == s2[j - 1] else 1
            matrix[i][j] = min(matrix[i - 1][j - 1] + delta,
                               matrix[i - 1][j] + 1,
                               matrix[i][j - 1] + 1)
    return DataFrame(matrix, list(' ' + s1), list(' ' + s2))
edit_distance_matrix('abc', 'bcde')

,,b,c,d,e
,0,1,2,3,4
a,1,1,2,3,4
b,2,1,2,3,4
c,3,2,1,2,3


#### 基于编辑距离的相似度百分比

In [5]:
def edit_distance(s1, s2):
    l1, l2 = len(s1) + 1, len(s2) + 1
    matrix = np.zeros((l1, l2), dtype=int)
    for i in range(l1):
        matrix[i, 0] = i
    for j in range(l2):
        matrix[0, j] = j
    for i in range(1, l1):
        for j in range(1, l2):
            delta = 0 if s1[i - 1] == s2[j - 1] else 1
            matrix[i, j] = min(matrix[i - 1, j - 1] + delta,
                               matrix[i - 1, j] + 1,
                               matrix[i, j - 1] + 1)
    return matrix[-1, -1]
edit_distance(poem1, poem2)

30

In [6]:
1 - edit_distance(poem1, poem2) / (len(poem1 + poem2) / 2)

0.8557692307692308

## 欧氏距离 余弦距离

In [7]:
word2id = {w: i for i, w in enumerate(set(ls1 + ls2))}
length = len(word2id)
vec1, vec2 = np.zeros(length), np.zeros(length)
for w in ls1:
    vec1[word2id[w]] += 1
for w in ls2:
    vec2[word2id[w]] += 1

#### 欧氏距离

In [8]:
np.linalg.norm(vec1 - vec2)  # np.sqrt(np.sum(np.square(vec1 - vec2)))

6.782329983125268

#### 余弦距离

In [9]:
1 - vec1 @ vec2 / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

0.031974483907780904

#### 调包实现

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
vec1, vec2 = CountVectorizer(token_pattern='.').fit_transform([poem1, poem2])
print(euclidean_distances(vec1, vec2))
print(cosine_distances(vec1, vec2))

[[6.78232998]]
[[0.03197448]]


# 最长公共子串
Longest Common Substring

In [11]:
def lcs_matrix(s1, s2):
    l1, l2 = len(s1), len(s2)
    matrix = [[0 for j in range(l2 + 1)] for i in range(l1 + 1)]
    for i in range(l1):
        for j in range(l2):
            if s1[i] == s2[j]:
                matrix[i + 1][j + 1] = matrix[i][j] + 1
    return DataFrame(matrix, list(' ' + s1), list(' ' + s2))
lcs_matrix('cdef', 'abcde')

,,a,b,c,d,e
,0,0,0,0,0,0
c,0,0,0,1,0,0
d,0,0,0,0,2,0
e,0,0,0,0,0,3
f,0,0,0,0,0,0


In [12]:
def lcs(s1, s2):
    l1, l2 = len(s1), len(s2)
    matrix = np.zeros((l1 + 1, l2 + 1), dtype=int)
    max_len = 0  # 最长匹配的长度
    p = 0  # 最长匹配对应在s1中的最后一位
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i] == s2[j]:
                matrix[i + 1, j+1] = matrix[i, j] + 1
                max_len, p = max([(max_len, p), (matrix[i + 1, j + 1], i + 1)])
    return s1[p - max_len: p]  # 返回最长子串及其长度
print(lcs(poem1, poem2))

，惟有饮者留其名。陈王昔时宴平乐，斗酒十千恣欢谑。主人何为言少钱，径须沽取对君酌。五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。


# 最长公共子序列
The longest common subsequence

In [13]:
s1, s2 = 'ab_abc', 'abc_bcd'
l1, l2 = len(s1), len(s2)
# 生成字符串长度+1的零矩阵，保存对应位置匹配的结果
m = [[0 for j in range(l2 + 1)] for i in range(l1 + 1)]
# 记录转移方向
d = [['' for j in range(l2 + 1)] for i in range(l1 + 1)]
for p1 in range(l1):
    for p2 in range(l2):
        # 字符匹配成功，则该位置的值为左上方的值加1
        if s1[p1] == s2[p2]:
            m[p1 + 1][p2 + 1] = m[p1][p2] + 1
            d[p1 + 1][p2 + 1] = 'ok'
        # 左值大于上值，则该位置的值为左值，并标记回溯时的方向
        elif m[p1 + 1][p2] > m[p1][p2 + 1]:
            m[p1 + 1][p2 + 1] = m[p1 + 1][p2]
            d[p1 + 1][p2 + 1] = '←'
        # 上值大于左值，则该位置的值为上值，并标记方向↑
        else:
            m[p1 + 1][p2 + 1] = m[p1][p2 + 1]
            d[p1 + 1][p2 + 1] = '↑'

In [14]:
DataFrame(m, list(' ' + s1), list(' ' + s2))

,,a,b,c,_,b,c,d
,0,0,0,0,0,0,0,0
a,0,1,1,1,1,1,1,1
b,0,1,2,2,2,2,2,2
_,0,1,2,2,3,3,3,3
a,0,1,2,2,3,3,3,3
b,0,1,2,2,3,4,4,4
c,0,1,2,3,3,4,5,5


In [15]:
DataFrame(d, list(' ' + s1), list(' ' + s2))

,,a,b,c,_,b,c,d
,,,,,,,,
a,,ok,←,←,←,←,←,←
b,,↑,ok,←,←,ok,←,←
_,,↑,↑,↑,ok,←,←,←
a,,ok,↑,↑,↑,↑,↑,↑
b,,↑,ok,↑,↑,ok,←,←
c,,↑,↑,ok,↑,↑,ok,←


In [16]:
def lcs(s1, s2):
    l1, l2 = len(s1), len(s2)
    # 生成字符串长度+1的零矩阵，保存对应位置匹配的结果
    m = np.zeros((l1 + 1, l2 + 1))
    # 记录转移方向
    d = np.empty_like(m, dtype=str)
    for i in range(l1):
        for j in range(l2):
            # 字符匹配成功，则该位置的值为左上方的值加1
            if s1[i] == s2[j]:
                m[i + 1, j + 1] = m[i, j] + 1
                d[i + 1, j + 1] = 'O'
            # 左值大于上值，则该位置的值为左值，并标记回溯时的方向
            elif m[i + 1, j] > m[i, j + 1]:
                m[i + 1, j + 1] = m[i + 1, j]
                d[i + 1, j + 1] = '←'
            # 上值大于左值，则该位置的值为上值，并标记方向↑
            else:
                m[i + 1, j + 1] = m[i, j + 1]
                d[i + 1, j + 1] = '↑'
    s = []
    while m[l1, l2]:  # 不为空时
        c = d[l1, l2]
        if c == 'O':  # 匹配成功，插入该字符，并向左上角找下一个
            s.append(s1[l1 - 1])
            l1 -= 1
            l2 -= 1
        if c == '←':  # 根据标记，向左找下一个
            l2 -= 1
        if c == '↑':  # 根据标记，向上找下一个
            l1 -= 1
    s.reverse()
    return ''.join(s)
print(lcs(poem1, poem2))

《》——李白君不见黄河之水天上来，奔流到海不复回。君不见明镜悲白发，朝如青暮成雪。人生得意须尽欢，莫使金樽空对月。天生有，千金散尽还复来。烹羊宰牛且为乐，会须一饮三百杯。岑夫子，丹丘生，与君一曲，请君为我倾。钟鼓玉足贵，但长醉不复醒。古来圣皆，惟有饮者留其名。陈王昔时宴平乐，斗酒十千恣欢谑。主人何为言少钱，径须沽取对君酌。五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。


# TfIdf文本相似度

#### 语料加载

In [17]:
with open('古诗.txt', encoding='utf-8') as f:
    seqs = f.read().split()
q_ls = [i.split('|')[1] for i in seqs]
a_ls = [i.split('|')[0] for i in seqs]

#### 训练tfidf向量转换器

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(token_pattern='.')
vectorizer.fit(q_ls)
vec1, vec2 = vectorizer.transform([poem1, poem2])
cosine_similarity(vec1, vec2)

array([[0.90681949]])

In [19]:
X = vectorizer.transform(q_ls)
def ask(q, n=1):
    q = vectorizer.transform([q])  # tfidf向量化
    indexs = cosine_similarity(X, q).reshape(-1)  # 余弦相似度
    indexs = np.argsort(-indexs)  # 按索引倒排
    return [a_ls[i] for i in indexs[:n]]
for _ in range(5):
    q = input('输入：').strip()
    for e, i in enumerate(ask(q)):
        print(e, i)

输入：请君为我倾耳听
0 将进酒(李白)
输入：钟鼓馔玉不足贵
0 将进酒(李白)
输入：但愿长醉不复醒
0 将进酒(李白)
输入：钟鼓玉帛岂足贵
0 惜樽空(李白)
输入：但用长醉不复醒
0 惜樽空(李白)


# 词向量文本相似度

In [20]:
from gensim.models import Word2Vec
ls_of_words = [list(i) for i in q_ls]
model = Word2Vec(ls_of_words)
w2i = {w: i for i, w in enumerate(model.wv.index2word, 1)}
vectors = np.concatenate((np.zeros((1, 100)), model.wv.vectors), axis=0)
w2v = lambda w: vectors[w2i.get(w, 0)]

In [21]:
vec1 = np.mean([w2v(w) for w in poem1], axis=0)
vec2 = np.mean([w2v(w) for w in poem2], axis=0)

In [22]:
print(cosine_similarity([vec1], [vec2]))
vec1 @ vec2 / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

[[0.99998519]]


0.9999851879273223